# Testing Shaplet Generation

In [1]:
import shaplet_util as util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
sample = pd.read_csv('../../data/144/13_treadmill_3mph_0%.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)

In [3]:
plt.close()
plt.plot(sample['x'])

In [4]:
walking = sample.iloc[3000:19000]
walking = walking.reset_index()

In [5]:
period = util.find_single_peak_dist(walking, window_size=5000, step=100)
walking.index.size

16000

In [6]:
plt.close()
plt.plot(period['upper'], ".")
plt.xlabel('Window Index')
plt.ylabel('Average Peak Distance')

In [8]:
period['upper'].median()

97.5

In [9]:
plt.close()
plt.plot(period['lower'], ".")
plt.xlabel('Window Index')
plt.ylabel('Average Peak Distance')

In [10]:
period['lower'].median()

66.0

In [11]:
shaplet_len = period['upper'].median()

In [12]:
def dist(row):
    return row.apply(row_dist, args=(row.name,))

def row_dist(column, row):
    test_shaplet =  walking.loc[int(row):int(row)+shaplet_len].reset_index()
    ts_slice = walking.loc[int(column):int(column)+shaplet_len].reset_index()
    return np.linalg.norm(test_shaplet['x']- ts_slice['x'])

def set_column(x):
    return x.fillna(x.name)

In [13]:
#df = pd.DataFrame(data=[], index=np.arange(0,1000,20), columns=np.arange(0,1000,1))
#df = df.apply(set_column, axis=0)

In [14]:
#df = df.apply(dist, axis=1)

In [15]:
walking_test=walking.iloc[0:200]
plt.close()
plt.plot(walking_test['x'])

In [16]:
df = pd.DataFrame(data=[], index=np.arange(0,walking_test['x'].index.size-shaplet_len,20), columns=walking_test.index)
df.index = df.index.map(int)

In [17]:
df = df.apply(set_column, axis=0)

In [18]:
euclidean_dist_df = df.apply(dist, axis=1)

In [19]:
def get_best(row):
    num_best = int(row.size * .1)
    sorted_row = row.sort_values().reset_index()
    return sorted_row.loc[0:num_best, row.name]
euclidean_dist_df.apply(get_best, axis=1)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.0,0.873175,0.940862,1.056421,1.374186,1.680600,1.805756,2.058748,2.339378,2.542740,...,2.914596,3.014927,3.034116,3.079105,3.094298,3.119425,3.152701,3.182022,3.183490,3.186641
20,0.0,0.871736,0.895335,1.005982,1.055875,1.398339,1.555582,1.646374,1.744347,1.911814,...,2.282540,2.398446,2.499230,2.761050,2.794012,2.805990,2.820885,2.836694,2.876791,2.940219
40,0.0,0.945395,0.945404,1.284001,1.436667,1.658862,1.815855,1.816194,1.968041,2.259498,...,2.559322,2.580707,2.829202,2.837156,2.843631,2.876979,2.924001,2.954335,3.033786,3.042693
60,0.0,0.943961,0.947337,1.290562,1.449289,1.657562,1.809069,1.821687,1.978459,2.254317,...,2.569393,2.585236,2.836447,2.913720,2.940561,2.975431,2.999629,3.019015,3.057462,3.065933
80,0.0,0.765448,0.781992,1.273812,1.278871,1.447738,1.530182,1.570121,1.590500,1.990299,...,2.088847,2.213086,2.419101,2.488053,2.623821,2.740424,2.756933,2.760199,2.793785,2.799136
100,0.0,0.882467,0.884167,1.695388,1.703413,2.395551,2.415136,2.914596,2.927097,2.939932,...,2.963341,2.992427,3.020255,3.023388,3.090493,3.160699,3.164075,3.239144,3.333413,3.340047


In [45]:
euclidean_dist_df.to_csv("144_candidate_shaplets")